# RECEIPT DETECTION OCR - RESEARCH PIPELINE
**Objective:** Komparasi arsitektur OCR (Tesseract, EasyOCR, PaddleOCR, MMOCR, DocTR) untuk ekstraksi data struk belanja dengan optimasi akurasi metrik absolut (CER/WER) pada level karakter dan F1-Score pada level *field* informasi.

---

### 1. SETUP & IMPORTS
*inisialisasi environment. Pastikan Runtime diatur ke GPU (T4/V100/A100) sebelum menjalankan.*

- **[Code]** Mount Google Drive untuk akses dataset.
- **[Code]** Install dependencies utama:
  `!apt install tesseract-ocr`
  `!pip install pytesseract easyocr paddleocr paddlepaddle-gpu python-doctr[tf] mmocr`
- **[Code]** Install dependencies evaluasi & NLP:
  `!pip install jiwer rapidfuzz opencv-python-headless matplotlib seaborn`
- **[Code]** Import libraries & set Random Seed untuk reproduktibilitas riset.

---

### 2. EDA & DATA CLEANING
*Fase validasi input. Model yang baik akan gagal jika diberi data kotor atau anotasi yang salah.*

- **[Markdown]** Analisis distribusi dataset (Rasio data mulus vs. data terdistorsi).
- **[Code]** Visualisasi sampel dataset Geometri (Skewed/Tilted), Pencahayaan (Low-light/Faded), dan Struktural (Cutted/Occluded).
- **[Code]** Deduplikasi menggunakan *Perceptual Hashing* (pHash) untuk membuang gambar identik.
- **[Code]** Pembagian dataset terspesifikasi (Stratified Split: 70% Train, 15% Val, 15% Test) dengan proporsi tingkat kesulitan yang merata.
- **[Code]** Standarisasi format Ground Truth ke dalam JSON tunggal.

---

### 3. PREPROCESSING EXPERIMENTS
*Implementasi percabangan preprocessing. Model Deep Learning (Branch A/B) vs Traditional (Branch C).*

- **[Markdown]** Metodologi Koreksi Geometri dan Pencahayaan.
- **[Code]** Fungsi Geometric Correction (Deskew & Perspective Transform menggunakan Canny Edge + Hough Lines).
- **[Code]** Fungsi Illumination Correction (CLAHE pada *color space* LAB).
- **[Code]** Binarization Test (Adaptive Gaussian Thresholding) - **Khusus untuk Tesseract**.
- **[Code]** Visualisasi komparatif: Original vs. Pipeline DL (Geometri+CLAHE) vs. Pipeline Tesseract (Geometri+Binarization).

---

### 4. OCR MODEL BENCHMARKING (INTI RESEARCH)
*Fase pengujian model secara terisolasi menggunakan data uji (Test Set) yang telah diproses.*

- **[Code]** Inisialisasi Class Wrapper untuk penyeragaman output format: `[{"box": [x,y,w,h], "text": "...", "conf": 0.9}]`.
- **[Code]** 4.1 Pipeline Tesseract (Input: Branch C Preprocessing)
- **[Code]** 4.2 Pipeline EasyOCR (Input: Branch A+B Preprocessing)
- **[Code]** 4.3 Pipeline PaddleOCR (Input: Branch A+B Preprocessing)
- **[Code]** 4.4 Pipeline MMOCR (Input: Branch A+B Preprocessing)
- **[Code]** 4.5 Pipeline DocTR (Input: Branch A+B Preprocessing)
- **[Code]** 4.6 Evaluasi Komparatif:
  - Kalkulasi metrik Word Error Rate (WER) & Character Error Rate (CER) menggunakan library `jiwer`.
  - Kalkulasi *Inference Time* (FPS).
  - Pembuatan Tabel Pandas dan visualisasi *Bar Chart* komparasi performa.

---

### 5. POST-PROCESSING & INFORMATION EXTRACTION
*Fase penerjemahan kotak teks acak menjadi informasi terstruktur (Key-Value).*

- **[Markdown]** Logika *Spatial Sorting* untuk rekonstruksi baris teks (Mengatasi teks miring minor).
- **[Code]** Implementasi Algoritma Y-Axis Tolerance Sorting (Pengelompokan baris berdasarkan delta $Y$).
- **[Code]** Implementasi Algoritma X-Axis Sorting (Pengurutan kiri-kanan per baris).
- **[Code]** Parsing JSON/Dict dari raw OCR text hasil pemenang tahap 4.
- **[Code]** Regex & Fuzzy matching ekstraksi informasi:
  - Ekstraksi **Tanggal** (Regex variasi format tanggal lokal/internasional).
  - Ekstraksi **Total Belanja** (Pencarian anchor "Total"/"Amount" dilanjut validasi format mata uang).
  - Ekstraksi **Merchant** (Ekstraksi baris teratas + `rapidfuzz` string matching ke database).
- **[Code]** Evaluasi presisi ekstraksi (Berapa persen field Total/Tanggal yang benar-benar akurat 100%?).

---

### 6. KESIMPULAN & EXPORT
*Pembungkusan seluruh riset menjadi modul yang siap diintegrasikan oleh tim Software Engineering.*

- **[Markdown]** Justifikasi pemilihan model final (Menyeimbangkan CER, Waktu Eksekusi, dan Akurasi Ekstraksi).
- **[Markdown]** Analisis Error (Pada kasus seperti apa pipeline ini masih gagal?).
- **[Code]** Penyatuan fungsi menjadi satu Class Pipeline end-to-end `class ReceiptScanner`.
- **[Code]** Export pipeline terbaik menjadi class Python (`.py`) atau model ter-pickle siap pakai.


In [ ]:
# ============================================================
# CELL 1 — SETUP DEPENDENCIES
# STATUS: Semua package di bawah sudah terinstall di .venv
#         (Python 3.12 | Kernel: "Python 3.12 (OCR Research)")
#
# Jalankan cell ini hanya jika environment direset / baru clone.
# ============================================================

import subprocess, sys

def pip_install(packages: str, extra_index: str = ""):
    cmd = [sys.executable, "-m", "pip", "install"] + packages.split()
    if extra_index:
        cmd += ["--index-url", extra_index]
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout[-2000:] if result.stdout else "")
    if result.returncode != 0:
        print("[ERROR]", result.stderr[-500:])
    return result.returncode

# ── Step 1: PyTorch + CUDA 12.1 (NVIDIA GPU) ────────────────
print("[1/3] Installing PyTorch CUDA 12.1 ...")
pip_install(
    "torch torchvision torchaudio",
    extra_index="https://download.pytorch.org/whl/cu121"
)

# ── Step 2: Core OCR engines ────────────────────────────────
# NOTE: MMOCR belum support Python 3.12 di Windows secara resmi.
print("[2/3] Installing OCR engines ...")
pip_install(
    "pytesseract easyocr paddleocr paddlepaddle-gpu "
    "python-doctr[torch]"
)

# ── Step 3: Evaluation & utilities ─────────────────────────
print("[3/3] Installing evaluation & utility packages ...")
pip_install(
    "jiwer rapidfuzz imagehash Pillow "
    "opencv-python matplotlib seaborn "
    "pandas scikit-learn tqdm ipykernel requests"
)

print("\n✅  Setup selesai. Restart kernel lalu jalankan Cell 2.")

In [ ]:
# ============================================================
# CELL 2 — IMPORT LIBRARIES & GLOBAL CONFIG
# ============================================================

# ── Standard Library ─────────────────────────────────────────
import os
import re
import json
import io
import time
import random
import warnings
from pathlib import Path

# ── Numerik & Visualisasi ────────────────────────────────────
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from tqdm import tqdm   # Menggunakan tqdm standar agar tidak kena error IProgress

# ── Networking ───────────────────────────────────────────────
import requests

# ── Image Processing ─────────────────────────────────────────
import cv2
from PIL import Image
import imagehash                        # pHash deduplication

# ── OCR Engines ──────────────────────────────────────────────
import pytesseract
# ⚠️  Sesuaikan path jika Tesseract diinstall di lokasi berbeda
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

import easyocr
from paddleocr import PaddleOCR
from doctr.io import DocumentFile
from doctr.models import ocr_predictor

# ⚠️  MMOCR: Belum support Python 3.12 + Windows secara resmi.
# from mmocr.apis import MMOCRInferencer

# ── Evaluation ───────────────────────────────────────────────
from jiwer import wer, cer
from rapidfuzz import fuzz, process
from sklearn.model_selection import train_test_split

# ── CUDA check ───────────────────────────────────────────────
import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️  Device  : {DEVICE}")
if DEVICE == "cuda":
    print(f"🎮  GPU     : {torch.cuda.get_device_name(0)}")
    print(f"💾  VRAM    : {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# ── Global Config ────────────────────────────────────────────
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if DEVICE == "cuda":
    torch.cuda.manual_seed_all(SEED)

warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8-whitegrid")
%matplotlib inline

print("\n✅  Semua library berhasil diimport.")

In [ ]:
# ============================================================
# CELL 3 — LOAD DATASET (LOCAL)
# Path otomatis dari lokasi notebook — tidak hardcoded username.
# ============================================================

PROJECT_ROOT = Path.cwd()
DATASET_DIR  = PROJECT_ROOT / "dataset"
IMG_DIR      = DATASET_DIR / "images"
ANNOT_DIR    = DATASET_DIR / "annotations"
GT_FILE      = ANNOT_DIR / "ground_truth.json"

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tiff"}

print(f"📂  Project root : {PROJECT_ROOT}")
print(f"📂  Dataset root : {DATASET_DIR}")

image_paths: list[Path] = []
if IMG_DIR.exists():
    image_paths = sorted(
        p for p in IMG_DIR.rglob("*") if p.suffix.lower() in IMG_EXTS
    )
else:
    print(f"⚠️  Folder gambar tidak ditemukan: {IMG_DIR}")

print(f"🖼️  Total gambar : {len(image_paths)}")

ground_truth: dict = {}
if GT_FILE.exists():
    with open(GT_FILE, "r", encoding="utf-8") as f:
        ground_truth = json.load(f)
    print(f"📋  Ground truth : {len(ground_truth)} entri dari {GT_FILE.name}")
else:
    print(f"⚠️  Ground truth belum ada — akan dibuat di Cell EDA.")

if image_paths:
    summary: dict[str, int] = {}
    for p in image_paths:
        summary[p.parent.name] = summary.get(p.parent.name, 0) + 1
    summary_df = pd.DataFrame(
        list(summary.items()), columns=["Kategori", "Jumlah Gambar"]
    ).sort_values("Jumlah Gambar", ascending=False).reset_index(drop=True)
    print("\n📊  Distribusi dataset per kategori:")
    display(summary_df)
else:
    print("\nℹ️  Dataset lokal kosong — gunakan Cell EDA untuk fetch dari HuggingFace.")

print("\n✅  Load dataset selesai.")

In [ ]:
# ============================================================
# CELL 4 — TAHAP 1: EDA — FETCH DATA DARI HUGGING FACE
# Dataset: naver-clova-ix/cord-v1
# ✅ Telah diuji di terminal — endpoint & field mapping verified
# ============================================================

BASE_URL = "https://datasets-server.huggingface.co"
DATASET  = "naver-clova-ix%2Fcord-v1"
FETCH_N  = 100   # jumlah sampel yang diambil

def fetch_hf_dataset(n: int = 100) -> list[dict]:
    """
    Fetch rows dari HuggingFace Datasets Server.
    /rows kadang 500 — otomatis fallback ke /first-rows.
    """
    print(f"Mengambil {n} baris dari HuggingFace API (CORD-v1)...")

    url = (
        f"{BASE_URL}/rows"
        f"?dataset={DATASET}&config=default&split=train"
        f"&offset=0&length={n}"
    )
    r = requests.get(url, timeout=30)

    if r.status_code != 200:
        print(f"⚠️  /rows gagal ({r.status_code}), fallback ke /first-rows...")
        url = f"{BASE_URL}/first-rows?dataset={DATASET}&config=default&split=train"
        r = requests.get(url, timeout=30)
        r.raise_for_status()

    return r.json().get("rows", [])


# ── Proses Ingesti Data ──────────────────────────────────────
raw_rows = fetch_hf_dataset(FETCH_N)
dataset_processed = []

if raw_rows:
    for item in tqdm(raw_rows, desc="Memproses Baris Data"):
        row_content = item["row"]

        # image field adalah dict: {'src': url, 'height': int, 'width': int}
        image_field = row_content["image"]
        image_url   = image_field["src"] if isinstance(image_field, dict) else image_field

        # ground_truth adalah JSON string
        gt_raw = row_content["ground_truth"]
        gt     = json.loads(gt_raw) if isinstance(gt_raw, str) else gt_raw

        dataset_processed.append({
            "image_url"    : image_url,
            "image_width"  : image_field.get("width",  0) if isinstance(image_field, dict) else 0,
            "image_height" : image_field.get("height", 0) if isinstance(image_field, dict) else 0,
            "ground_truth" : gt,
            "label"        : "clear",
        })

    print(f"\n✅  Berhasil memuat {len(dataset_processed)} sampel.")

    # ── Visualisasi Sampel EDA ───────────────────────────────
    sample = dataset_processed[0]
    img_bytes   = requests.get(sample["image_url"], timeout=15).content
    img_preview = Image.open(io.BytesIO(img_bytes))

    plt.figure(figsize=(6, 8))
    plt.imshow(img_preview)
    plt.title("Sampel Dataset CORD-v1")
    plt.axis("off")
    plt.show()

    # ── Analisis Struktur Ground Truth ──────────────────────
    gt_keys = list(sample["ground_truth"].keys())
    print(f"📋  Field GT yang ditemukan : {gt_keys}")
    print(f"📐  Dimensi gambar          : {sample['image_width']}×{sample['image_height']} px")

else:
    print("❌  Tidak ada data yang berhasil diproses.")

In [ ]:
# ============================================================
# CELL 5 — TAHAP 2: DATA CLEANING (pHASH DEDUP & SPLIT)
# ✅ Telah diuji di terminal — logika verified
# ============================================================

print("Memulai Tahap 2: Data Cleaning & Stratified Splitting...")
print("-" * 55)

final_clean = []
seen_hashes: dict = {}
dup_count   = 0

# ── 1. Deduplikasi Visual (Perceptual Hashing) ───────────────
# Hamming Distance <= 2 dianggap duplikat visual
for sample in tqdm(dataset_processed, desc="Deduplikasi pHash"):
    try:
        img_bytes = requests.get(sample["image_url"], timeout=10).content
        img_obj   = Image.open(io.BytesIO(img_bytes))
        h         = imagehash.phash(img_obj)

        if any((h - k) <= 2 for k in seen_hashes):
            dup_count += 1
        else:
            seen_hashes[h] = True
            final_clean.append(sample)
    except Exception:
        continue  # skip gambar yang gagal diload

print(f"\nHasil Cleaning:")
print(f"   - Duplikat dihapus : {dup_count}")
print(f"   - Dataset unik     : {len(final_clean)}")

# ── 2. Stratified Split 70 / 15 / 15 ────────────────────────
if len(final_clean) >= 10:
    train_data, temp_data = train_test_split(
        final_clean, test_size=0.30, random_state=SEED
    )
    val_data, test_data = train_test_split(
        temp_data, test_size=0.50, random_state=SEED
    )

    print(f"\nStatistik Pembagian Dataset:")
    print(f"   - Training Set   : {len(train_data)}")
    print(f"   - Validation Set : {len(val_data)}")
    print(f"   - Testing Set    : {len(test_data)}")
else:
    print("⚠️  Data tidak mencukupi untuk splitting (min. 10 sampel).")

print("-" * 55)
print("✅  Tahap 2 Selesai. Dataset siap untuk Preprocessing Experiments.")